# MPS canonical forms 

In [20]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:

import numpy as np
from numpy import linalg as LA
from tensornetwork import ncon
import logging

import math 
import copy 

from myMPSstuff import myMPS, randMPS, bringCan, checkIdMatrix, expValOneSite


logging.basicConfig()
logger = logging.getLogger('Something')
logger.setLevel(logging.INFO)

In [96]:
test = myMPS()

mpstest = np.random.rand(4,7,3,7)

test = myMPS(mpstest)

test.getIndices()

test.updatePhysIndices([9,9,9,9])

test.updateEdgeIndices([999,999])

test.getIndices()

test.getNorm()


MPS with length 7 and physical d=2
chi [1, 5, 5, 5, 5, 5, 5, 1]
MPS with length 4 and physical d=3
chi [7, 7, 7, 7, 7]
Indices:

[[29, 1, 30], [30, 2, 31], [31, 3, 32], [32, 4, 33]] [[29, 1, 38], [38, 2, 39], [39, 3, 40], [40, 4, 33]]


array(2322960.58995492)

## Are the SVs the same in the Left and Right forms ?

In [79]:
mymps2 = myMPS(randMPS(20))

bringCan(mymps2,mode='LR',epsTrunc=1e-8)
aaa = mymps2.SV

bringCan(mymps2,mode='L')
bbb = mymps2.SV

#print(aaa)

#print(bbb)

diff = [aaa[i]-bbb[i] for i in range(0,len(aaa)) ]

#[ aa[np.abs(aa) < 1e-14] = 0  for aa in diff]

#[ [el if el > 1e-14 else 0 for el in aa]  for aa in diff]

print("######")
#print([ [el if el > 1e-14 else 0 for el in aa]  for aa in diff])
print( np.all([ [aa < 1e-14 for aa in el] for el in diff]) )
print("######")


# looks like YES 

MPS with length 20 and physical d=2
chi [1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1]
Chis = [1, 2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1]
Chis = [1, 2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2, 1]
Chis = [1, 2, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 2, 1]
######
[True]
######


/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


## Check that we're actually in proper L-R canon forms by contracting a bit and see if we recover identity matrices and normalizations 

In [99]:
mymps3 = myMPS(randMPS(6))
aaa = bringCan(mymps3,mode='LR',epsTrunc=1e-6)
bbb = bringCan(mymps3,mode='L')

# So now the MPS should be normalized and in  LEFT canonical form 
# Check by contracting some A's 

print(f"Norm={mymps3.getNorm()}")

mp = mymps3.MPS
sv = mymps3.SV
svinv = [svv**(-1) for svv in sv ]

for m in mp:
    checkIdMatrix(ncon([m,np.conj(m)],[[1,2,-1],[1,2,-2]]))



# Looks good 

# Now try and build the B's using the recipe 
#Bj = (L_{j-1})^-1 @ Aj @ Lj


print("")

for j in range(0,len(mp)):
    #print(j,np.shape(svinv[j]),  np.shape(mp[j]), np.shape(sv[j+1]))
    mb = ncon([np.diag(svinv[j]),  mp[j] ,  np.diag(sv[j+1]) ], [[-1,1],[1,-2,2],[2,-3]])
    checkIdMatrix(ncon([mb,np.conj(mb)],[[-1,1,2],[-2,1,2]]))


# Also looks good. 



MPS with length 6 and physical d=2
chi [1, 5, 5, 5, 5, 5, 1]
Chis = [1, 2, 4, 5, 5, 5, 1]
Chis = [1, 2, 4, 5, 4, 2, 1]
Chis = [1, 2, 4, 5, 4, 2, 1]
Indices:

[[43, 1, 44], [44, 2, 45], [45, 3, 46], [46, 4, 47], [47, 5, 48], [48, 6, 49]] [[43, 1, 56], [56, 2, 57], [57, 3, 58], [58, 4, 59], [59, 5, 60], [60, 6, 49]]
Norm=1.0
identity, size = 2
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 2
identity, size = 1

identity, size = 1
identity, size = 2
identity, size = 4
identity, size = 5
identity, size = 4
identity, size = 2


## Try computing an expectation value, first the "hard" way (ncon the whole thing), then using the right-canonical expression

In [ ]:
# Try to do it a bit better, define separately psi and psi*
# and compute the norm 

testL = 7
testd = 2
testChi = 5

mypsi = myMPS(randMPS(testL,chi=testChi, d=testd), offIndices=3)
mypsic = myMPS(randMPS(testL,chi=testChi, d=testd), offIndices=5)


# Now define the operator to exp-value

# Say we wanna compute <sigma_x> 
op1 = np.array([[0,1.],[1.,0]])

# at site r = 3 

psiOpens = [1,2,66,4,5,6,7]
psicOpens = [1,2,67,4,5,6,7]
op1Opens = [[66,67]]

mypsi.updateIndices(psiOpens)
mypsic.updateIndices(psicOpens)

mtocon = copy.deepcopy(mypsi.MPS)
mtocon.extend(mypsic.MPS)
mtocon.extend([op1])

print(len(mtocon),len(mypsi.MPS),len(mypsic.MPS),len(op1))

itocon = mypsi.getIndices()
itocon.extend(mypsic.getIndices())
itocon.extend(op1Opens)

#print(itocon)
#print(mtocon)
#print(len(itocon),len(mtocon))

tnc = ncon(mtocon,itocon)

print(np.real_if_close(tnc/norm))


# Now compute it with the canonical form 

copymypsi = copy.deepcopy(mypsi)
psiR = bringCan(copymypsi,mode='LR',epsTrunc=1e-12)


conTen = [np.diag(copymypsi.SV[1]),np.diag(copymypsi.SV[1]),copymypsi.MPS[2],np.conj(copymypsi.MPS[2]),op1]
conIdx = [[1,2],[1,3],[3,4,5],[2,6,5],[4,6]]

[print(np.shape(elem)) for elem in conTen]
print(len(conTen),len(conIdx))

print(np.real_if_close(ncon(conTen,conIdx)))

copymypsi2 = copy.deepcopy(mypsi)

print(expValOneSite(copymypsi2,op1,2))

MPS with length 7 and physical d=2
chi [5, 5, 5, 5, 5, 5]
MPS with length 7 and physical d=2
chi [5, 5, 5, 5, 5, 5]
[{'vL': 0, 'ph': 1, 'vR': 22}, {'vL': 22, 'ph': 2, 'vR': 23}, {'vL': 23, 'ph': 3, 'vR': 24}, {'vL': 24, 'ph': 4, 'vR': 25}, {'vL': 25, 'ph': 5, 'vR': 26}, {'vL': 26, 'ph': 6, 'vR': 27}, {'vL': 27, 'ph': 7, 'vR': 0}]
[[1, 22], [22, 2, 23], [23, 3, 24], [24, 4, 25], [25, 5, 26], [26, 6, 27], [27, 7]]
[[1, 36], [36, 2, 37], [37, 3, 38], [38, 4, 39], [39, 5, 40], [40, 6, 41], [41, 7]]
[[1, 22], [22, 2, 23], [23, 3, 24], [24, 4, 25], [25, 5, 26], [26, 6, 27], [27, 7], [1, 36], [36, 2, 37], [37, 3, 38], [38, 4, 39], [39, 5, 40], [40, 6, 41], [41, 7]]
norm/alized = 164429410.63905928,1.0
15 7 7 2
0.9761630417692106
Chis = [2, 4, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 4, 2]
(4, 4)
(4, 4)
(4, 2, 5)
(4, 2, 5)
(2, 2)
5 5
0.9761630417692098
Chis = [2, 4, 5, 5, 5, 5]
Chis = [2, 4, 5, 5, 4, 2]
0.9761630417692098


# The canonical form Gamma-Lambda 

Let's build it from our LR sweep
and check that we get the same results for our MPS 